# EECS759P Coursework 1
- Name: Bheki Maenetja
- Student ID: 230382466

## Imports

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict, deque
import math
import hashlib
import string
import random

## Plotting Functions

## Loading Data

In [2]:
# Function provided in undirected_map.py
def load_data(df):
    station_dict = defaultdict(list)
    zone_dict = defaultdict(dict)

    # get data row by row
    for index, row in df.iterrows():
        start_station = row[0]
        end_station = row[1]

        line = row[2] 
        
        act_cost = int(row[3])
        
        zone1 = row[4]
        zone2 = row[5]

        # station dictionary of child station tuples (child_name, cost from parent to the child)
        # {"Mile End": [("Stepney Green", 2), ("Wembley", 1)]}
        station_list = station_dict[start_station]
        station_list.append((end_station, line, act_cost))

        # the following two lines add the other direction of the tube "step"
        station_list = station_dict[end_station]
        station_list.append((start_station, line, act_cost))

        # we add the main zone
        zone_dict[start_station]["main"] = zone1
        # we add the secondary zone

        if zone2 != "0":
            zone_dict[start_station]["2nd"] = zone2
            # if the secondary zone is not 0 it's the main zone for the ending station
            zone_dict[end_station]["main"] = zone2
        else:
            # otherwise the main zone for the ending station is the same as for the starting station
            zone_dict[end_station]["main"] = zone1

    return station_dict, zone_dict

In [3]:
tube_df = pd.read_csv('tubedata.csv', header=None)
tube_df.head()

,0,1,2,3,4,5
0,Harrow & Wealdstone,Kenton,Bakerloo,3,5,0
1,Kenton,South Kenton,Bakerloo,2,4,0
2,South Kenton,North Wembley,Bakerloo,2,4,0
3,North Wembley,Wembley Central,Bakerloo,2,4,0
4,Wembley Central,Stonebridge Park,Bakerloo,3,4,0


In [4]:
stations, zones = load_data(tube_df)

In [5]:
set(tube_df[0])

{'Acton Town',
 'Aldgate',
 'Aldgate East',
 'Alperton',
 'Angel',
 'Archway',
 'Arnos Grove',
 'Arsenal',
 'Baker Street',
 'Balham',
 'Bank/Monument',
 'Barbican',
 'Barking',
 'Barkingside',
 'Barons Court',
 'Bayswater',
 'Becontree',
 'Belsize Park',
 'Bermondsey',
 'Bethnal Green',
 'Blackfriars',
 'Blackhorse Road',
 'Bond Street',
 'Borough',
 'Boston Manor',
 'Bounds Green',
 'Bow Road',
 'Brent Cross',
 'Bromley-by-Bow',
 'Buckhurst Hill',
 'Burnt Oak',
 'Caledonian Road',
 'Camden Town',
 'Canada Water',
 'Canary Wharf',
 'Canning Town',
 'Cannon Street',
 'Canons Park',
 'Chalfont & Latimer',
 'Chalk Farm',
 'Chancery Lane',
 'Charing Cross',
 'Chesham',
 'Chigwell',
 'Chiswick Park',
 'Chorleywood',
 'Clapham Common',
 'Clapham North',
 'Clapham South',
 'Cockfosters',
 'Colindale',
 'Colliers Wood',
 'Covent Garden',
 'Croxley',
 'Dagenham East',
 'Dagenham Heathway',
 'Debden',
 'Dollis Hill',
 'Ealing Broadway',
 'Ealing Common',
 "Earls' Court",
 'East Acton',
 'East F

In [6]:
set(tube_df[4])

{'1', '2', '3', '4', '5', '6', 'a', 'b', 'c', 'd'}

In [7]:
stations

defaultdict(list,
            {'Harrow & Wealdstone': [('Kenton', 'Bakerloo', 3)],
             'Kenton': [('Harrow & Wealdstone', 'Bakerloo', 3),
              ('South Kenton', 'Bakerloo', 2)],
             'South Kenton': [('Kenton', 'Bakerloo', 2),
              ('North Wembley', 'Bakerloo', 2)],
             'North Wembley': [('South Kenton', 'Bakerloo', 2),
              ('Wembley Central', 'Bakerloo', 2)],
             'Wembley Central': [('North Wembley', 'Bakerloo', 2),
              ('Stonebridge Park', 'Bakerloo', 3)],
             'Stonebridge Park': [('Wembley Central', 'Bakerloo', 3),
              ('Harlesden', 'Bakerloo', 2)],
             'Harlesden': [('Stonebridge Park', 'Bakerloo', 2),
              ('Willesden Junction', 'Bakerloo', 2)],
             'Willesden Junction': [('Harlesden', 'Bakerloo', 2),
              ('Kensal Green', 'Bakerloo', 3)],
             'Kensal Green': [('Willesden Junction', 'Bakerloo', 3),
              ("Queen's Park", 'Bakerloo', 3)],
 

In [8]:
zones

defaultdict(dict,
            {'Harrow & Wealdstone': {'main': '5'},
             'Kenton': {'main': '4'},
             'South Kenton': {'main': '4'},
             'North Wembley': {'main': '4'},
             'Wembley Central': {'main': '4'},
             'Stonebridge Park': {'main': '3'},
             'Harlesden': {'main': '3'},
             'Willesden Junction': {'main': '3'},
             'Kensal Green': {'main': '2'},
             "Queen's Park": {'main': '2'},
             'Kilburn Park': {'main': '2'},
             'Maida Vale': {'main': '2'},
             'Warwick Avenue': {'main': '2'},
             'Paddington': {'main': '1'},
             'Edgware Road': {'main': '1'},
             'Marylebone': {'main': '1'},
             'Baker Street': {'main': '1'},
             "Regent's Park": {'main': '1'},
             'Oxford Circus': {'main': '1'},
             'Piccadilly Circus': {'main': '1'},
             'Charing Cross': {'main': '1'},
             'Embankment': {'main': '1'},


## 2.0 Node Class

In [9]:
class Node:
    def __init__(self, data, parent=None):
        self.name = data[0]
        self.line = data[1]
        self.cost = data[2]
        self.parent = parent

    def __str__(self):
        return f"{self.name} via {self.line} (t = {self.cost})"

    def __repr__(self):
        return self.__str__()

## 2.1 DFS, BFS and UCS

### Building optimal path to goal node

In [10]:
def build_path(node):
    path = [node]
    path_cost = node.cost
    
    while node.parent:
        node = node.parent
        path = [node] + path
        path_cost = path_cost if node.cost is None else path_cost + node.cost

    path_str = f"{path[0].name} -> " + " -> ".join([n.__str__() for n in path[1:]])
    print(f"PATH: {path_str}")
    print(f"\nPath cost (travel time): {path_cost} minutes")
    
    return path, path_cost

### Depth-First Search

In [11]:
def dfs_search(start, goal):
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    start_node = Node((start, None, None), None)
    
    frontier = [
        Node(s, start_node) 
        for s in stations[start].copy()
    ]
    
    explored = [start]
    num_explored = 0
    print(f"Start: {start} —>")
    
    while frontier:
        node = frontier.pop()
        num_explored += 1

        print(f"{node} ->")
        
        # Goal check
        if node.name == goal:
            print(f"GOAL: {node.name}")
            print(f"\nGoal found!\nNumber of explorations = {num_explored}")
            return node
        
        # Node expansion
        new_nodes = stations[node.name].copy()
        
        for n in new_nodes:
            if n[0] not in explored:
                n_obj = Node(n, node)
                frontier.append(n_obj)
                explored.append(n[0])

In [12]:
# goal_node = dfs_search("Euston", "Victoria")
goal_node = dfs_search("Whitechapel", "Westminster")

Start: Whitechapel —>
Stepney Green via Hammersmith & City (t = 3) ->
Mile End via District (t = 2) ->
Bow Road via District (t = 1) ->
Bromley-by-Bow via District (t = 2) ->
West Ham via District (t = 2) ->
Canning Town via Jubilee (t = 3) ->
North Greenwich via Jubilee (t = 3) ->
Canary Wharf via Jubilee (t = 3) ->
Canada Water via Jubilee (t = 3) ->
Bermondsey via Jubilee (t = 2) ->
London Bridge via Jubilee (t = 3) ->
Borough via Northern (t = 2) ->
Elephant & Castle via Northern (t = 1) ->
Kennington via Northern (t = 2) ->
Oval via Northern (t = 2) ->
Stockwell via Northern (t = 3) ->
Brixton via Victoria (t = 2) ->
Vauxhall via Victoria (t = 3) ->
Pimlico via Victoria (t = 1) ->
Victoria via Victoria (t = 3) ->
Green Park via Victoria (t = 2) ->
Oxford Circus via Victoria (t = 2) ->
Warren Street via Victoria (t = 2) ->
Goodge Street via Northern (t = 2) ->
Euston via Northern (t = 1) ->
King's Cross St. Pancras via Northern (t = 2) ->
Highbury & Islington via Victoria (t = 4) -

In [13]:
goal_node

Westminster via Jubilee (t = 3)

In [14]:
path, path_cost = build_path(goal_node)

PATH: Whitechapel -> Stepney Green via Hammersmith & City (t = 3) -> Mile End via District (t = 2) -> Bow Road via District (t = 1) -> Bromley-by-Bow via District (t = 2) -> West Ham via District (t = 2) -> Canning Town via Jubilee (t = 3) -> North Greenwich via Jubilee (t = 3) -> Canary Wharf via Jubilee (t = 3) -> Canada Water via Jubilee (t = 3) -> Bermondsey via Jubilee (t = 2) -> London Bridge via Jubilee (t = 3) -> Borough via Northern (t = 2) -> Elephant & Castle via Northern (t = 1) -> Kennington via Northern (t = 2) -> Oval via Northern (t = 2) -> Stockwell via Northern (t = 3) -> Vauxhall via Victoria (t = 3) -> Pimlico via Victoria (t = 1) -> Victoria via Victoria (t = 3) -> Green Park via Victoria (t = 2) -> Westminster via Jubilee (t = 3)

Path cost (travel time): 49 minutes


In [15]:
path

[Whitechapel via None (t = None),
 Stepney Green via Hammersmith & City (t = 3),
 Mile End via District (t = 2),
 Bow Road via District (t = 1),
 Bromley-by-Bow via District (t = 2),
 West Ham via District (t = 2),
 Canning Town via Jubilee (t = 3),
 North Greenwich via Jubilee (t = 3),
 Canary Wharf via Jubilee (t = 3),
 Canada Water via Jubilee (t = 3),
 Bermondsey via Jubilee (t = 2),
 London Bridge via Jubilee (t = 3),
 Borough via Northern (t = 2),
 Elephant & Castle via Northern (t = 1),
 Kennington via Northern (t = 2),
 Oval via Northern (t = 2),
 Stockwell via Northern (t = 3),
 Vauxhall via Victoria (t = 3),
 Pimlico via Victoria (t = 1),
 Victoria via Victoria (t = 3),
 Green Park via Victoria (t = 2),
 Westminster via Jubilee (t = 3)]

In [16]:
path_cost

49

### Breadth-First Search

In [17]:
def bfs_search(start, goal):    
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    start_node = Node((start, None, None), None)
    
    frontier = [
        Node(s, start_node)
        for s in stations[start].copy()
    ]
    
    explored = [start]
    num_explored = 0
    print(f"Start: {start} —>")
    
    while frontier:
        node = frontier.pop(0)
        num_explored += 1

        print(f"{node} ->")
        
        # Goal check
        if node.name == goal:
            print(f"GOAL: {node.name}")
            print(f"\nGoal found!\nNumber of explorations = {num_explored}")
            return node

        # Node expansion
        new_nodes = stations[node.name].copy()
        
        for n in new_nodes:
            if n[0] not in explored:
                n_obj = Node(n, node)
                frontier.append(n_obj)
                explored.append(n[0])

In [18]:
# goal_node = bfs_search("Euston", "Victoria")
goal_node = bfs_search("Whitechapel", "Westminster")

Start: Whitechapel —>
Aldgate East via District (t = 2) ->
Stepney Green via District (t = 3) ->
Shoreditch via East London (t = 2) ->
Shadwell via East London (t = 2) ->
Aldgate East via Hammersmith & City (t = 2) ->
Stepney Green via Hammersmith & City (t = 3) ->
Tower Hill via District (t = 2) ->
Liverpool Street via Hammersmith & City (t = 4) ->
Mile End via District (t = 2) ->
Wapping via East London (t = 1) ->
Aldgate via Circle (t = 4) ->
Bank/Monument via Circle (t = 2) ->
Aldgate East via District (t = 2) ->
Bethnal Green via Central (t = 3) ->
Moorgate via Circle (t = 2) ->
Stratford via Central (t = 4) ->
Stepney Green via District (t = 2) ->
Bow Road via District (t = 1) ->
Shadwell via East London (t = 1) ->
Rotherhithe via East London (t = 1) ->
St. Paul's via Central (t = 2) ->
Cannon Street via Circle (t = 2) ->
London Bridge via Northern (t = 2) ->
Waterloo via Waterloo & City (t = 4) ->
Barbican via Circle (t = 2) ->
Old Street via Northern (t = 1) ->
Leyton via Centr

In [19]:
goal_node

Westminster via Jubilee (t = 2)

In [20]:
path, path_cost = build_path(goal_node)

PATH: Whitechapel -> Aldgate East via District (t = 2) -> Tower Hill via District (t = 2) -> Bank/Monument via Circle (t = 2) -> Waterloo via Waterloo & City (t = 4) -> Westminster via Jubilee (t = 2)

Path cost (travel time): 12 minutes


In [21]:
path

[Whitechapel via None (t = None),
 Aldgate East via District (t = 2),
 Tower Hill via District (t = 2),
 Bank/Monument via Circle (t = 2),
 Waterloo via Waterloo & City (t = 4),
 Westminster via Jubilee (t = 2)]

In [22]:
path_cost

12

### Uniform Cost Search

In [23]:
def ucs_search(start, goal):    
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    start_node = Node((start, None, None), None)
    
    frontier = [
        Node(s, start_node)
        for s in stations[start].copy()
    ]
    frontier.sort(key=lambda x: x.cost)
    
    explored = [start]
    num_explored = 0

    print(f"START: {start} —>")
    
    while frontier:
        node = frontier.pop(0)
        num_explored += 1

        print(f"{node} ->")
        
        # Goal check
        if node.name == goal:
            print(f"GOAL: {node.name}")
            print(f"\nGoal found!\nNumber of explorations = {num_explored}")
            return node
        
        # Node expansion
        new_nodes = stations[node.name].copy()
        
        for n in new_nodes:
            if n[0] not in explored:
                n_obj = Node(n, node)
                frontier.append(n_obj)
                explored.append(n[0])

        frontier.sort(key=lambda x: x.cost)

In [24]:
goal_node = ucs_search("Euston", "Victoria")
# goal_node = ucs_search("Whitechapel", "Westminster")

START: Euston —>
Warren Street via Northern (t = 1) ->
Warren Street via Victoria (t = 1) ->
Mornington Crescent via Northern (t = 2) ->
Camden Town via Northern (t = 1) ->
Mornington Crescent via Northern (t = 1) ->
King's Cross St. Pancras via Northern (t = 2) ->
King's Cross St. Pancras via Victoria (t = 2) ->
Goodge Street via Northern (t = 2) ->
Tottenham Court Road via Northern (t = 1) ->
Leicester Square via Northern (t = 1) ->
Covent Garden via Piccadilly (t = 1) ->
Oxford Circus via Victoria (t = 2) ->
Bond Street via Central (t = 1) ->
Marble Arch via Central (t = 1) ->
Kentish Town via Northern (t = 2) ->
Chalk Farm via Northern (t = 2) ->
Euston Square via Circle (t = 2) ->
Angel via Northern (t = 2) ->
Russell Square via Piccadilly (t = 2) ->
Warren Street via Northern (t = 2) ->
Holborn via Central (t = 2) ->
Chancery Lane via Central (t = 1) ->
Charing Cross via Northern (t = 2) ->
Embankment via Bakerloo (t = 1) ->
Piccadilly Circus via Piccadilly (t = 2) ->
Regent's Pa

In [25]:
goal_node

Victoria via Victoria (t = 2)

In [26]:
path, path_cost = build_path(goal_node)

PATH: Euston -> Warren Street via Northern (t = 1) -> Oxford Circus via Victoria (t = 2) -> Green Park via Victoria (t = 2) -> Victoria via Victoria (t = 2)

Path cost (travel time): 7 minutes


In [27]:
path

[Euston via None (t = None),
 Warren Street via Northern (t = 1),
 Oxford Circus via Victoria (t = 2),
 Green Park via Victoria (t = 2),
 Victoria via Victoria (t = 2)]

In [28]:
path_cost

7

## 2.2 Comparison of DFS, BFS and UCS (report question)

## 2.3 Extending the Cost Function

In [29]:
def ucs_search_2(start, goal, penalty=2):    
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    start_node = Node((start, None, None), None)
    
    frontier = [
        Node(s, start_node)
        for s in stations[start].copy()
    ]
    frontier.sort(key=lambda x: x.cost)
    
    explored = [start]
    num_explored = 0

    # input(f"Starting frontier: {frontier} >>> ")
    print(f"START: {start} —>")
    
    while frontier:
        # input(f"\n\n===== NEW ITERATION =====\n\n")
        node = frontier.pop(0)
        num_explored += 1

        # input(f"Current node: {node} >>> ")
        print(f"{node} ->")
        
        # Goal check
        if node.name == goal:
            print(f"GOAL: {node.name}")
            print(f"\nGoal found!\nNumber of explorations = {num_explored}")
            return node

        
        # Node expansion
        # new_nodes = stations[node.data[0]].copy()
        new_nodes = [
            (c[0], c[1], c[2])
            if c[1] == node.line
            else (c[0], c[1], c[2] + penalty) # applying penalty for line change
            for c in stations[node.name].copy()
        ]
        # input(f"\nProspective children: {new_nodes} >>> ")
        # input(f"\nProspective children (with penalty applied): {new_nodes_2} >>> ")
        
        for n in new_nodes:
            if n[0] not in explored:
                n_obj = Node(n, node)
                frontier.append(n_obj)
                explored.append(n[0])

        # input(f"\nNew frontier: {frontier} >>> ")
        frontier.sort(key=lambda x: x.cost)
        # input(f"\nNew sorted frontier: {frontier} >>> ")

In [30]:
goal_node = ucs_search_2("Whitechapel", "Westminster", 2)
# goal_node = ucs_search_2("Euston", "Victoria")

START: Whitechapel —>
Aldgate East via District (t = 2) ->
Shoreditch via East London (t = 2) ->
Shadwell via East London (t = 2) ->
Wapping via East London (t = 1) ->
Shadwell via East London (t = 1) ->
Rotherhithe via East London (t = 1) ->
Canada Water via East London (t = 1) ->
Aldgate East via Hammersmith & City (t = 2) ->
Tower Hill via District (t = 2) ->
Surrey Quays via East London (t = 2) ->
Aldgate East via District (t = 2) ->
Stepney Green via District (t = 3) ->
Mile End via District (t = 2) ->
Bow Road via District (t = 1) ->
Stepney Green via District (t = 2) ->
Bromley-by-Bow via District (t = 2) ->
West Ham via District (t = 2) ->
Plaistow via District (t = 2) ->
Upton Park via District (t = 2) ->
East Ham via District (t = 2) ->
Stepney Green via Hammersmith & City (t = 3) ->
Bermondsey via Jubilee (t = 4) ->
London Bridge via Jubilee (t = 3) ->
Southwark via Jubilee (t = 2) ->
Waterloo via Jubilee (t = 1) ->
Westminster via Jubilee (t = 2) ->
GOAL: Westminster

Goal 

In [31]:
goal_node

Westminster via Jubilee (t = 2)

In [32]:
path, path_cost = build_path(goal_node)

PATH: Whitechapel -> Shadwell via East London (t = 2) -> Wapping via East London (t = 1) -> Rotherhithe via East London (t = 1) -> Canada Water via East London (t = 1) -> Bermondsey via Jubilee (t = 4) -> London Bridge via Jubilee (t = 3) -> Southwark via Jubilee (t = 2) -> Waterloo via Jubilee (t = 1) -> Westminster via Jubilee (t = 2)

Path cost (travel time): 17 minutes


In [33]:
path

[Whitechapel via None (t = None),
 Shadwell via East London (t = 2),
 Wapping via East London (t = 1),
 Rotherhithe via East London (t = 1),
 Canada Water via East London (t = 1),
 Bermondsey via Jubilee (t = 4),
 London Bridge via Jubilee (t = 3),
 Southwark via Jubilee (t = 2),
 Waterloo via Jubilee (t = 1),
 Westminster via Jubilee (t = 2)]

In [34]:
path_cost

17

## 2.4 Heuristic Search

### Heuristic Function

In [35]:
def get_lines(station):
    return {s[1] for s in stations[station]}

def heuristic(current, goal):
    if current.name == goal:
        return 0

    utility = 1
    lines = get_lines(goal)
    zone_maps = {
        "1": 1,
        "2": 2,
        "3": 3,
        "4": 4,
        "5": 5,
        "6": 6,
        "a": 7,
        "b": 8,
        "c": 9,
        "d": 10,
    }

    current_zone = zone_maps[zones[current.name]["main"]]
    goal_zone = zone_maps[zones[goal]["main"]]

    zone_dist = abs(current_zone - goal_zone)
    utility = utility + zone_dist + int(not current.line in lines)

    # print("\n\n===== NEW NODE =====\n\n")
    # print(f"Node: {current}")
    # print(f"Current zone: {current_zone}")
    # print(f"Goal zone: {goal_zone}")
    # print(f"Utility = {utility}")
    # print(f"Same line? {current.line in lines}")
    # print("\n====================")
    return utility
    

### Best-First Search

In [36]:
def best_first_search(start, goal):    
    if start == goal:
        print("Start and goal are the same!!!")
        return None

    start_node = Node((start, None, None), None)
    
    frontier = [
        Node(s, start_node)
        for s in stations[start].copy()
    ]
    # frontier.sort(key=lambda x: x.cost)
    # print(f"Frontier: {frontier}")
    frontier.sort(key=lambda x: heuristic(x, goal))
    
    explored = [start]
    num_explored = 0

    # print(f"Sorted frontier: {frontier}")
    # return
    print(f"START: {start} —>")
    
    while frontier:
        node = frontier.pop(0)
        num_explored += 1

        print(f"{node} ->")
        
        # Goal check
        if node.name == goal:
            print(f"GOAL: {node.name}")
            print(f"\nGoal found!\nNumber of explorations = {num_explored}")
            return node
        
        # Node expansion
        new_nodes = stations[node.name].copy()
        
        for n in new_nodes:
            if n[0] not in explored:
                n_obj = Node(n, node)
                frontier.append(n_obj)
                explored.append(n[0])

        # frontier.sort(key=lambda x: x.cost)
        frontier.sort(key=lambda x: heuristic(x, goal))

In [37]:
# goal_node = best_first_search("Euston", "Victoria")
goal_node = best_first_search("Whitechapel", "Westminster")
# goal_node = best_first_search("Upminster Bridge", "Edgware")

START: Whitechapel —>
Aldgate East via District (t = 2) ->
Tower Hill via District (t = 2) ->
Aldgate via Circle (t = 4) ->
Bank/Monument via Circle (t = 2) ->
Aldgate East via District (t = 2) ->
Cannon Street via Circle (t = 2) ->
Mansion House via Circle (t = 1) ->
Blackfriars via Circle (t = 1) ->
Temple via Circle (t = 1) ->
Embankment via Circle (t = 2) ->
Westminster via Circle (t = 2) ->
GOAL: Westminster

Goal found!
Number of explorations = 11


In [38]:
path, path_cost = build_path(goal_node)

PATH: Whitechapel -> Aldgate East via District (t = 2) -> Tower Hill via District (t = 2) -> Bank/Monument via Circle (t = 2) -> Cannon Street via Circle (t = 2) -> Mansion House via Circle (t = 1) -> Blackfriars via Circle (t = 1) -> Temple via Circle (t = 1) -> Embankment via Circle (t = 2) -> Westminster via Circle (t = 2)

Path cost (travel time): 15 minutes


In [39]:
path

[Whitechapel via None (t = None),
 Aldgate East via District (t = 2),
 Tower Hill via District (t = 2),
 Bank/Monument via Circle (t = 2),
 Cannon Street via Circle (t = 2),
 Mansion House via Circle (t = 1),
 Blackfriars via Circle (t = 1),
 Temple via Circle (t = 1),
 Embankment via Circle (t = 2),
 Westminster via Circle (t = 2)]

In [40]:
path_cost

15

## 3.1 Genetic Algorithm

### Starter Functions

In [41]:
MY_USERNAME = "ecs23496"

def get_password(student_username, l=10):
    # Possible characters include upper-case English letters, numbers between 0 and 9 (inclusive), 
    # and the underscore symbol
    options = string.digits + string.ascii_uppercase  + "_"

    h = hashlib.sha256(("ECS759P-AI"+student_username).encode("utf-8"))
    d = h.digest()
    s = ""
    for n in d:
      s += options[n%len(options)]

    return s[0:l]

# TO DO: replace *** with your EECS username and uncomment the code
student_password = get_password(MY_USERNAME)
print(student_password)

# Distance function
def distance_function(string_one, string_two):
    score = 0
    for i, j in zip(string_one, string_two):
        # Square of the absolute difference between two Unicode codes
        score += math.sqrt(abs(ord(i) - ord(j)))
    return score


# Upper bound of the distance value
MAX_VALUE = distance_function('0000000000', '__________')

# Compute normalised fitness for a list of candidate passwords 
def get_normalised_fitness(list_of_phrases, student_password):
    ordered_dict = dict()
    phrase_to_find = student_password
    for phrase in list_of_phrases:
        # Return 1 when a candidate matches the true password (string distance between them is zero)
        ordered_dict[phrase] = 1 - distance_function(phrase, phrase_to_find) / MAX_VALUE
    return ordered_dict

# Example of how to get fitness values for a list of candidates
get_normalised_fitness(['2Q4HHHHOTJ', '2HHZQYUOTJ'], student_password)

ZS09_JG55Z


{'2Q4HHHHOTJ': 0.48234584062986685, '2HHZQYUOTJ': 0.32453125283957984}

### Implementation

In [42]:
def initialise_population(pop_size):
    options = string.digits + string.ascii_uppercase  + "_"
    return ["".join(random.sample(options, 10)) for _ in range(pop_size)]

def assign_fitness(population, target):
    return get_normalised_fitness(population, target)

def mating_selection():
    pass

def cross_over(cross_pool, cross_prob=0.5):
    offspring = []
    other_parents = cross_pool.copy()
    
    for parent in cross_pool:
        rand_prob = random.random()
        if rand_prob <= cross_prob and parent in other_parents and len(other_parents) > 1:
            other_parents.remove(parent)
            other_parent = random.choice(other_parents)
            other_parents.remove(other_parent)
            cross_point = random.randint(0,10)

            child_1 = parent[:cross_point] + other_parent[cross_point:]
            child_2 = other_parent[:cross_point] + parent[cross_point:]

            offspring.append(child_1)
            offspring.append(child_2)

    return offspring

def mutation(mut_pool, mut_prob=0.5):
    mut_children = []
    
    for child in mut_pool:
        child = list(child)
        for i, letter in enumerate(child):
            rand_prob = random.random()
            if rand_prob <= mut_prob:
                randIdx = random.randint(0, len(child) - 1)
                temp = child[randIdx]
                child[randIdx] = child[i]
                child[i] = temp

        mut_children.append("".join(child))

    return mut_children

def selection():
    pass

def genetic_algorithm(target, n_gen=1000, pop_size=1000, cross_prob=0.5, mut_prob=0.5):
    half = pop_size // 2
    generations = n_gen
    
    # Initialise population
    new_pop = initialise_population(pop_size)

    if any(w == target for w in new_pop):
        print("Got it!!!")
        return
    
    # Assign fitness
    fitness = assign_fitness(new_pop, target)

    # Mating Selection
    new_pop.sort(key=lambda x: fitness[x], reverse=True)
    print(f"Fittest initial individual: {new_pop[0]} (f={fitness[new_pop[0]]})")
    
    for i in range(n_gen):
        mating_pool = new_pop.copy()[:half]
        
        # Crossover
        mating_pool = cross_over(mating_pool, cross_prob)

        # Mutation
        mating_pool = mutation(mating_pool, mut_prob)

        # Selection
        new_pop = new_pop.copy()[:half] + mating_pool
        fitness = assign_fitness(new_pop, target)
        new_pop.sort(key=lambda x: fitness[x], reverse=True)

        if fitness[new_pop[0]] == 1:
            generations = i + 1
            break

    print(f"Fittest final individual: {new_pop[0]} (f={fitness[new_pop[0]]}); found after {generations} generations.")
    print(f"Actual target: {target}")

In [43]:
genetic_algorithm(student_password)

Fittest initial individual: SRA9ZJO64E (f=0.7167895866173265)
Fittest final individual: ZS09_JG55Z (f=1.0); found after 871 generations.
Actual target: ZS09_JG55Z


In [44]:
genetic_algorithm(student_password, n_gen=1000, pop_size=1000, cross_prob=0.5, mut_prob=0.5)

Fittest initial individual: SOFBUJ913X (f=0.6489244186664747)
Fittest final individual: ZS09_JG55Z (f=1.0); found after 766 generations.
Actual target: ZS09_JG55Z
